# LISTA 9 - EJERCICIOS SQLITE
En estos ejercicios trabajaremos con la base de datos `shop.db` que crearemos a continuación. Utilizaremos la librería `sqlite3` para interactuar con la base de datos.

### Creación de Base de Datos Local
1.  No es necesario instalar ningún servidor. SQLite crea la base de datos en un archivo local.
2.  Vamos a trabajar con un archivo llamado `shop.db`.
3.  Podemos crear las tablas y cargar datos ejecutando scripts SQL o directamente desde Python.
4.  Para este ejemplo, asumiremos que vamos a crear la base de datos y las tablas desde cero o conectar a una existente.
import sqlite3

In [1]:
import sqlite3

# Función para inicializar la base de datos con datos de prueba
def crear_base_datos_prueba():
    try:
        conexion = sqlite3.connect('shop.db')
        cursor = conexion.cursor()
        
        # =====================================================
        # Tabla CATEGORIES (Categorías de productos)
        # =====================================================
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS categories (
                category_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL
            )
        """)
        
        # =====================================================
        # Tabla PRODUCTS (Productos)
        # =====================================================
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS products (
                product_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                price REAL,
                category_id INTEGER,
                FOREIGN KEY (category_id) REFERENCES categories (category_id)
            )
        """)
        
        # =====================================================
        # Tabla USERS (Usuarios)
        # =====================================================
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
                user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT NOT NULL UNIQUE,
                email TEXT NOT NULL UNIQUE,
                first_name TEXT,
                last_name TEXT,
                created_at TEXT DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # =====================================================
        # Tabla ORDERS (Pedidos/Compras)
        # =====================================================
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS orders (
                order_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER NOT NULL,
                order_date TEXT DEFAULT CURRENT_TIMESTAMP,
                total_items INTEGER DEFAULT 0,
                total_cost REAL DEFAULT 0.0,
                FOREIGN KEY (user_id) REFERENCES users(user_id)
            )
        """)
        
        # =====================================================
        # Tabla ORDER_ITEMS (Detalle de productos por pedido)
        # =====================================================
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS order_items (
                item_id INTEGER PRIMARY KEY AUTOINCREMENT,
                order_id INTEGER NOT NULL,
                product_id INTEGER NOT NULL,
                quantity INTEGER NOT NULL DEFAULT 1,
                unit_price REAL NOT NULL,
                subtotal REAL NOT NULL,
                FOREIGN KEY (order_id) REFERENCES orders(order_id),
                FOREIGN KEY (product_id) REFERENCES products(product_id)
            )
        """)
        
        # Limpiar tablas para evitar duplicados si se ejecuta varias veces
        cursor.execute("DELETE FROM order_items")
        cursor.execute("DELETE FROM orders")
        cursor.execute("DELETE FROM users")
        cursor.execute("DELETE FROM products")
        cursor.execute("DELETE FROM categories")
        
        # =====================================================
        # Insertar datos en CATEGORIES
        # =====================================================
        categorias = [
            (1, 'Electronics'),
            (2, 'Books'),
            (3, 'Clothing'),
            (4, 'Toys')
        ]
        cursor.executemany("INSERT INTO categories (category_id, name) VALUES (?, ?)", categorias)
        
        # =====================================================
        # Insertar datos en PRODUCTS
        # =====================================================
        productos = [
            (1, 'Smartphone', 599.99, 1),
            (2, 'Laptop', 999.99, 1),
            (3, 'Python Book', 29.99, 2),
            (4, 'T-Shirt', 19.99, 3),
            (5, 'Action Figure', 14.99, 4)
        ]
        cursor.executemany("INSERT INTO products (product_id, name, price, category_id) VALUES (?, ?, ?, ?)", productos)
        
        # =====================================================
        # Insertar datos en USERS
        # =====================================================
        usuarios = [
            ('jperez', 'juan.perez@email.com', 'Juan', 'Pérez'),
            ('agarcia', 'ana.garcia@email.com', 'Ana', 'García'),
            ('mlopez', 'maria.lopez@email.com', 'María', 'López'),
            ('csanchez', 'carlos.sanchez@email.com', 'Carlos', 'Sánchez')
        ]
        cursor.executemany("""
            INSERT INTO users (username, email, first_name, last_name) 
            VALUES (?, ?, ?, ?)
        """, usuarios)
        
        # =====================================================
        # Insertar datos en ORDERS
        # =====================================================
        pedidos = [
            (1,),  # Usuario 1 (Juan)
            (1,),  # Usuario 1 (Juan) - segundo pedido
            (2,),  # Usuario 2 (Ana)
            (3,),  # Usuario 3 (María)
        ]
        cursor.executemany("INSERT INTO orders (user_id) VALUES (?)", pedidos)
        
        # =====================================================
        # Insertar datos en ORDER_ITEMS
        # =====================================================
        # Obtener los IDs de los pedidos insertados
        cursor.execute("SELECT order_id FROM orders ORDER BY order_id")
        order_ids = [row[0] for row in cursor.fetchall()]
        
        items = [
            # Pedido 1: Juan compra 2 Smartphones y 1 Laptop
            (order_ids[0], 1, 2, 599.99, 2 * 599.99),
            (order_ids[0], 2, 1, 999.99, 1 * 999.99),
            # Pedido 2: Juan compra 3 Python Books
            (order_ids[1], 3, 3, 29.99, 3 * 29.99),
            # Pedido 3: Ana compra 1 Smartphone y 2 Laptops
            (order_ids[2], 1, 1, 599.99, 1 * 599.99),
            (order_ids[2], 2, 2, 999.99, 2 * 999.99),
            # Pedido 4: María compra 1 Smartphone
            (order_ids[3], 1, 1, 599.99, 1 * 599.99),
        ]
        cursor.executemany("""
            INSERT INTO order_items (order_id, product_id, quantity, unit_price, subtotal) 
            VALUES (?, ?, ?, ?, ?)
        """, items)
        
        # Actualizar totales en orders
        cursor.execute("""
            UPDATE orders SET 
                total_items = (SELECT SUM(quantity) FROM order_items WHERE order_items.order_id = orders.order_id),
                total_cost = (SELECT SUM(subtotal) FROM order_items WHERE order_items.order_id = orders.order_id)
        """)
        
        conexion.commit()
        print("Base de datos 'shop.db' creada e inicializada con datos de prueba.")
        print("\nTablas creadas:")
        print("  - categories: Categorías de productos")
        print("  - products: Productos de la tienda")
        print("  - users: Usuarios registrados")
        print("  - orders: Pedidos/Compras")
        print("  - order_items: Detalle de productos por pedido")
        
    except sqlite3.Error as e:
        print(f"Error al crear la base de datos: {e}")
    finally:
        if conexion:
            conexion.close()

# Ejecutar la función para crear la BD
crear_base_datos_prueba()

Base de datos 'shop.db' creada e inicializada con datos de prueba.

Tablas creadas:
  - categories: Categorías de productos
  - products: Productos de la tienda
  - users: Usuarios registrados
  - orders: Pedidos/Compras
  - order_items: Detalle de productos por pedido


## EJERCICIO 1: Conexión y Consulta Simple
Conéctate a la base de datos `shop.db`.
Crea un cursor.
Ejecuta una consulta `SELECT` para obtener todos los registros de la tabla `products`.
Muestra los resultados por pantalla iterando sobre el cursor o usando `fetchall()`.
No olvides cerrar la conexión al final.

In [2]:
conexion = sqlite3.connect('shop.db')
cursor = conexion.cursor()

cursor.execute("SELECT * FROM products")
productos = cursor.fetchall()

for producto in productos:
    print(producto)

conexion.close()

(1, 'Smartphone', 599.99, 1)
(2, 'Laptop', 999.99, 1)
(3, 'Python Book', 29.99, 2)
(4, 'T-Shirt', 19.99, 3)
(5, 'Action Figure', 14.99, 4)


## EJERCICIO 2: Filtrado de Datos
Conéctate a la base de datos `shop.db`.
Obtén y muestra los nombres y precios de los productos que cuesten más de 100 unidades monetarias.
Utiliza una consulta SQL con la cláusula `WHERE`.

In [3]:
conexion = sqlite3.connect('shop.db')
cursor = conexion.cursor()

cursor.execute("SELECT name, price FROM products WHERE price > 100")
productos = cursor.fetchall()

for producto in productos:
    print(producto)

conexion.close()

('Smartphone', 599.99)
('Laptop', 999.99)


## EJERCICIO 3: Inserción de Datos
Conéctate a la base de datos.
Inserta un nuevo producto en la tabla `products`.
Datos del producto: `product_id` = 100, `name` = 'Tablet', `price` = 299.99, `category_id` = 1.
Recuerda hacer `commit()` para guardar los cambios.
Verifica la inserción haciendo un `SELECT` del nuevo producto.

In [5]:
conexion = sqlite3.connect('shop.db')
cursor = conexion.cursor()

cursor.execute("""
    INSERT INTO products (product_id, name, price, category_id)
    VALUES (?, ?, ?, ?)
""", (100, "Tablet", 299.99, 1))

conexion.commit()

cursor.execute("SELECT * FROM products WHERE product_id = 100")
productos = cursor.fetchall()

for producto in productos:
    print(producto)

conexion.close()

(100, 'Tablet', 299.99, 1)


## EJERCICIO 4: Actualización de Datos
El precio de la 'Tablet' ha bajado.
Actualiza el registro del producto con `product_id` = 100 para que su precio sea 249.99.
Verifica el cambio.

In [6]:
conexion = sqlite3.connect('shop.db')
cursor = conexion.cursor()

cursor.execute("""
    UPDATE products
    SET price = ?
    WHERE product_id = 100
""", (249.99,))

conexion.commit()

cursor.execute("SELECT * FROM products WHERE product_id = 100")
productos = cursor.fetchall()

for producto in productos:
    print(producto)

conexion.close()

(100, 'Tablet', 249.99, 1)


## EJERCICIO 5: Eliminación de Datos
Elimina el producto 'Tablet' (id 100) de la base de datos.
Verifica que ya no existe.

In [7]:
conexion = sqlite3.connect('shop.db')
cursor = conexion.cursor()

cursor.execute("DELETE FROM products WHERE product_id = ?", (100,))

conexion.commit()

cursor.execute("SELECT * FROM products WHERE product_id = 100")
productos = cursor.fetchall()

for producto in productos:
    print(producto)

conexion.close()

## EJERCICIO 6: Consultas con GROUP BY
Queremos saber cuántos productos hay en cada categoría.
Realiza una consulta `SELECT` que cuente los productos agrupados por `category_id`.
Muestra el `category_id` y la cantidad de productos.

In [8]:
conexion = sqlite3.connect('shop.db')
cursor = conexion.cursor()

cursor.execute("""
    SELECT category_id, COUNT(*) as total_productos
    FROM products
    GROUP BY category_id
""")
consulta = cursor.fetchall()

for fila in consulta:
    print(fila)

conexion.close()

(1, 2)
(2, 1)
(3, 1)
(4, 1)


## EJERCICIO 7: Creación de Tablas y Relaciones
Crea una nueva tabla llamada `customers` con las siguientes columnas:
*   `customer_id`: Entero, Clave Primaria.
*   `first_name`: Texto, No Nulo.
*   `last_name`: Texto, No Nulo.
*   `email`: Texto, Único.

Inserta al menos 2 clientes en la tabla.

In [9]:
conexion = sqlite3.connect('shop.db')
cursor = conexion.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS customers (
        customer_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT UNIQUE
    )
""")

clientes = [
    (1, "Alba", "Aro", "alba@gmail.com"),
    (2, "Blanca", "Negro", "blanca@gmail.com")
]

cursor.executemany("""
    INSERT INTO customers (customer_id, first_name, last_name, email)
    VALUES (?, ?, ?, ?)
""", clientes)

conexion.commit()

cursor.execute("SELECT * FROM customers")
consulta = cursor.fetchall()

for fila in consulta:
    print(fila)

conexion.close()

(1, 'Alba', 'Aro', 'alba@gmail.com')
(2, 'Blanca', 'Negro', 'blanca@gmail.com')


## EJERCICIO 8: Funciones de Agregación
Cuenta cuántos productos hay en total en la tabla `products`.
Calcula el precio promedio de los productos.
Muestra ambos resultados.

In [10]:
conexion = sqlite3.connect('shop.db')
cursor = conexion.cursor()

cursor.execute("""
    SELECT COUNT(*) as total_productos, AVG(price) as precio_promedio
    FROM products
""")
consulta = cursor.fetchall()

for fila in consulta:
    print(fila)

conexion.close()

(5, 332.99)
